In [1]:
import numpy as np
np.seterr(over='ignore')
import cv2
import sys
import math

img = cv2.imread("in_img.jpg")
F_İmage = np.zeros(img.shape)

sigma_i = 12    #sigma i value
sigma_s = 17    #sigma s value

i = 0
while i < len(img):     #Loop for each row
    print ("Loading... %" +str(round(((i)/len(img))*100))+  "  Please wait. It may take several minutes.")
    j = 0
    while j < len(img[0]):   # Loop for each column
        
        filtered_list =[0,0,0]   # I define filtered list to holds filtered RGB values for each pixels 
        Wp_0, Wp_1, Wp_2 = 0,0,0 
        half_lenght = 7//2  # Half lenght of filter lenght   filter lenght = 7
        k=0
        while k < 7:   # Loop for each x neighbour
            l = 0
            while l < 7:  # Loop for each y neighbour
                if (i - (half_lenght - k)) < len(img):  # edges control
                    n_x = i - (half_lenght - k)           # if neighbours are out of image this alghoritm dont takes out of pixels
                if (j - (half_lenght - l)) < len(img[0]):  # edges control
                    n_y = j - (half_lenght - l)            #takes neighbour only if they are inside the image.           
                         
                density_difference_0=img[n_x][n_y][0] - img[i][j][0]  # intensity differences between neighbours and selected pixel.
                density_difference_1=img[n_x][n_y][1] - img[i][j][1]                               
                density_difference_2=img[n_x][n_y][2] - img[i][j][2]
                # gaussian with distances 
                gs = (1.0 / (2 * math.pi * (sigma_s ** 2))) * math.exp(- (np.sqrt((n_x-i)**2 + (n_y-j)**2) ** 2) / (2 * sigma_s ** 2))
                # gaussian with instensity differences for each RGB pixels.
                w_1 = (1.0 / (2 * math.pi * (sigma_i ** 2))) * math.exp(- (density_difference_0 ** 2) / (2 * sigma_i ** 2)) * gs
                w_2 = (1.0 / (2 * math.pi * (sigma_i ** 2))) * math.exp(- (density_difference_1 ** 2) / (2 * sigma_i ** 2)) * gs
                w_3 = (1.0 / (2 * math.pi * (sigma_i ** 2))) * math.exp(- (density_difference_2 ** 2) / (2 * sigma_i ** 2)) * gs
                 
                
                filtered_list[0] += img[n_x][n_y][0] * w_1
                filtered_list[1] += img[n_x][n_y][1] * w_2
                filtered_list[2] += img[n_x][n_y][2] * w_3
                
                Wp_0 += w_1
                Wp_1 += w_2
                Wp_2 += w_3
                
                l += 1
            k += 1
        filtered_list[0] = filtered_list[0] / Wp_0
        filtered_list[1] = filtered_list[1] / Wp_1
        filtered_list[2] = filtered_list[2] / Wp_2
        
        m = 0
        while m < 3:
            F_İmage[i][j][m]= int(round(filtered_list[m]))  # Round each RGB pixels's intensity values and assign each of them to image_list
            m += 1
        
        j += 1
    i += 1
cv2.imwrite("F_İmage_my.jpg", F_İmage)
filtered_image_OpenCV = cv2.bilateralFilter(img, 7, 12, 17)
cv2.imwrite("OpenCv_filtered_image.jpg", filtered_image_OpenCV)

#The purpose of bileteral filter is smoothing the image without blurring edges.
#Because of that i tryed severel sigma values from 2 to 200 for each of sigma(sigma_i and sigma_s)
#First i fixed the sigma_s value to 10 and increase sigma_i value 5 by 5 and when i get closer to higher values to 100
# image blured too much. I understand that if i increase sigma_s value too much image get blured too much but edges also blured and goes bad.
# my aim is smooth image but not smooth edges. So i understand that i must take sigma_s value a bit low not 50 or 100.
#Second i fixed the sigma_i value to 10 and increase sigma_s value 5 by 5 and i can't see a big visible change from it.
#But i increase sigma_s and sigma_i values simultaneasly smoothnes of image increase and also edges are more sharp.
#As a result when sigma_s value increasing smoothness sigma_i value protecting the sharpness of the edges.
# I combine two values to find the nice smoothing image and i select sigma_i 12 and sigma_s 17 for best result.


Loading... %0  Please wait. It may take several minutes.
Loading... %0  Please wait. It may take several minutes.
Loading... %0  Please wait. It may take several minutes.


KeyboardInterrupt: 